In [1]:
from fastai.text.all import *
import pandas as pd

### if using google colab ###
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!wget https://drive.google.com/uc?export=download&id=18rSp1jWkoTJsaOG6BUrSDi9AFnJNtUfl
df = pd.read_csv('df_preprocessed.csv', quoting=csv.QUOTE_ALL, encoding='utf-8')
df.head()

--2023-03-21 17:48:08--  https://drive.google.com/uc?export=download
Resolving drive.google.com (drive.google.com)... 74.125.24.100, 74.125.24.139, 74.125.24.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.24.100|:443... connected.
HTTP request sent, awaiting response... 400 Bad Request
2023-03-21 17:48:09 ERROR 400: Bad Request.



,Unnamed: 0,Text,OpenStatus
0,0,"For Mongodb is it better to reference an object or use a natural String key? I am building a corpus of indexed sentences in different languages. I have a collection of Languages which have both an ObjectId and the ISO code as a key. Is it better to use a reference to the Language collection or store a key like ""en"" or ""fr""? I suppose it's a compromise between: - ease of referencing the Language - object in that collection - speed in doing queries where the sentence has a certain language - the size of the data on disk Any best practices that I should know of? mongodb",open
1,1,"How to insert schemalocation in a xml document via DOM i create a xml document with JAXP and search a way to insert the schemalocation. At the moment my application produces: <?xml version=""1.0"" encoding=""UTF-8""?> <root> ... </root> But i need: <?xml version=""1.0"" encoding=""UTF-8""?> <root xmlns=""namespaceURL"" xmlns:xs=""http://www.w3.org/2001/XMLSchema-instance"" xs:schemaLocation=""namespaceURL pathToMySchema.xsd""> ... </root> My code: StreamResult result = new StreamResult(writer); Document doc = getDocument(); Transformer trans = transfac.newTransformer(); trans.setOutputProperty(OutputKey...",open
2,2,"Too many lookup tables What are the adverse effects of having too many lookup tables in the database? I have to incorportate too many Enumerations, based on the applications. What would experts advice? sql-server database-design enums",open
3,3,"What is this PHP code in VB.net I am looking for the vb.net equivalent of this function: I have no experience of PHP but came across this code which apparently gives you a random image url from google images function GetRandomImageURL($topic='', $min=0, $max=100) { // get random image from Google if ($topic=='') $topic='image'; $ofs=mt_rand($min, $max); $geturl='http://www.google.ca/images?q=' . $topic . '&start=' . $ofs . '&gbv=1'; $data=file_get_contents($geturl); $f1='<div id=""center_col"">'; $f2='<a href=""/imgres?imgurl='; $f3='&amp;imgrefurl='; $pos1=strpos($data, $f1)+strlen($f1); if ...",too localized
4,4,"Spring-Data mongodb querying multiple classes stored in the same collection With Spring-Data, you can use the @Document annotation to specify which collection to save the object to. Say i have two classes, Student and Teacher, both persisted into the people collection. When I execute the following code: mongo.find(new Query(), Teacher.class); result contains both Student and Teacher. Looking in the data created by Spring-Data, each document contains a ""_class"" field which indicate which class it is persisted from. This is field not used in find as an filter to return only Teacher? How do I...",open


In [19]:
dls = TextDataLoaders.from_df(df, text_col='Text', label_col='OpenStatus', valid_pct=0.1)

In [20]:
dls.show_batch()

,text,category
0,xxbos xxmaj fix xxmaj memory xxmaj leak xxmaj crash xxmaj log [ link to xxmaj xxunk xxmaj post][1 ] [ 1 ] : http : / / stackoverflow.com / questions / xxunk / help - fix - memory - leak - release t h r e a d 3 c r a s h e d : 0 l i b o b j c . a . d y l i b 0 x 3 3 a 0 6 4 6 6 0 x 3 3 a 0 3 xxwrep 3 0 + 1 3 4 1 4 1 a v f o u n d a t i o n 0 x 3 1 a e 7 4 1 e 0 x 3 1 a b 8 xxwrep 3 0 + 1 9 3 5 6 6 2 l i b s y s t e m .,not a real question
1,"xxbos xxmaj firefox on a xxmaj mac crashes when i try to use xxup html table to xxmaj excel export tool i have search a lot to find a xxup html to xxmaj excel js export . xxmaj and i have found one that i like and works pretty well except on a xxup mac . xxmaj it crashes if i try to use it in xxmaj firefox . xxmaj in xxmaj safari is okey . xxmaj this is the code : < ! doctype xxup html xxup public "" - / / xxup w3c / / xxup dtd xxup html 4.01 / / xxup en "" "" http : / / xxrep 3 w .w3.org / xxup tr / html4 / strict.dtd "" > < html lang=""en "" > < head > < title > html to xxmaj excel export < / title > < script type=""text / javascript",off topic
2,"xxbos php login script for my custom php site xxmaj please help me with an application i m building . i m new to php and i m hoping that learning through experience is the best way to do it . xxmaj here is my situation … .. i have a mysql database which hae "" members "" table with my username & xxunk , this seems to be connecting with no xxunk . in addition i have the following pages and xxunk running on them : xxrep 90 _ index.php - where my form is < ! doctype html xxup public "" - / / xxup w3c / / xxup dtd xxup xhtml 1.0 xxmaj transitional / / xxup en "" "" http : / / xxrep 3 w .w3.org / xxup tr / xhtml1 / xxup dtd / xhtml1-transitional.dtd "" > < html xmlns=""http : / / xxrep",not constructive
3,"xxbos xxmaj could someone explain these code snippets ? xxmaj in the following code , could someone explain to me how the following works ? / * * xxunk * * xxmaj sniffer example of xxup tcp / xxup ip packet capture using libpcap . * * xxmaj version xxunk ( 2005 - 07 - 05 ) * xxmaj copyright ( c ) 2005 xxmaj the xxmaj tcpdump xxmaj group * * xxmaj this software is intended to be used as a practical example and * demonstration of the libpcap library ; available at : * http : / / xxrep 3 w xxunk / * xxrep 76 * * * xxmaj this software is a modification of xxmaj tim xxmaj xxunk ' "" xxunk "" * demonstration source code , released as follows : * * xxunk * xxmaj copyright ( c ) 2002 xxmaj tim xxmaj xxunk *",not constructive
4,"xxbos error in wordpress syntax error , unexpected xxup t_string in / xxrep 3 * / xxunk on line 1 i am upload my website and i see this error : syntax error , unexpected xxup t_string in / xxrep 3 * / xxunk on line 1 xxmaj what must i do ? is there anybody to help me ? plz and the xxunk cod is here < ? php / * xxrep 83 - * / / * xxmaj post xxmaj thumbnail xxmaj support / * xxrep 83 - * / xxunk - xxunk ( xxunk - xxunk ( xxunk ' ) ) { xxunk ' , 472 , 280 , xxunk ' , 193 , 138 , true ) ; } / / xxmaj load languages xxunk ' , xxunk ( ) . ' / xxunk = xxunk = xxup xxunk . "" / languages / $ xxunk",too localized
5,"xxbos xxmaj oracle query performance issue i have a table discussion of which structure is following : xxmaj table name : discussion , xxmaj i d name parent_id root_id … other columns 1 xxunk 0 0 2 xxunk 1 1 3 xxunk 2 1 4 xxunk 3 1 5 xxunk 4 1 i d was defined as xxup pk , parent_id and root_id has been indexed . these rows of table was built as a hierarchical xxunk of xxunk ) , please note i have a column root_id which is used to describe which rows are in the same tree . i written two sql to get the tree : * * sql 1 * * xxup select * from reply start with ( parent_id=0 xxup and id= ? ) connect by prior id = parent_id * * sql 2 * * xxup select * xxup from ( xxup select",open
6,"xxbos xxup sql xxmaj server 2012 installation failure xxmaj an error occurred for a dependency of the feature causing the setup process f

In [21]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [22]:
epochs, lr = 4, 1e-2
learn.fine_tune(epochs, lr)

epoch,train_loss,valid_loss,accuracy,time
0,1.129363,1.032778,0.605261,07:18


epoch,train_loss,valid_loss,accuracy,time
0,0.981550,0.932113,0.643473,17:22
1,0.897849,0.865420,0.672346,17:20
2,0.839689,0.837319,0.688030,17:21
3,0.814117,0.837897,0.689242,17:22


In [23]:
learn.show_results()
learn.save('high_level_API')

,text,category,category_
0,"xxbos xxmaj post xxmaj creator 2.3 xxmaj plugin xxmaj not xxmaj working xxmaj properly xxmaj in xxmaj word xxmaj press ? xxmaj hi i am working in word press plugin , that plugin name is post creator 2.3 , using this we have to post several post at a time .. the problem is its posting only 150 posts instead of 1 xxrep 3 0 posts .. also i referred lot of things , i do n't know to solve .. ! [ enter image description here][2 ] ! [ enter image description here][3 ] [ 2 ] : http : / / i.stack.imgur.com / xxunk [ 3 ] : http : / / i.stack.imgur.com / xxunk i am new to word press , i do n't know how to solve this problem , anybody help to solve this problem . < ? php function xxunk ( ) { xxunk",not a real question,not a real question
1,"xxbos xxmaj slow broker response time in activemq 5.6 xxmaj we have a setup where user generates report using xxup soap over activemq with xxmaj jboss 5.1.0 as the xxup as . xxmaj our project is migrating from activemq 4 to activemq 5.6 and we have noticed with the default configuration , the time of generation of one of the reports has increased by 50 secs . xxmaj this particular report consists of the maximum amount of data among all others , and the other reports are all showing response times close to our earlier setup . xxmaj so we are suspecting it has to do with some setting wherein the message queue is not able to handle larger data . xxmaj we have checked and the time from database to return the query result is identical , and this extra 50 sec is coming in activemq . xxmaj here",not a real question,open
2,"xxbos paypal xxmaj subscription xxup ipn - xxup php i created the following paypal xxup ipn page to handle transactions . xxmaj this is my first time . xxmaj for some reason every transaction triggers the very last part : else if ( strcmp ( $ res , "" invalid "" ) = = 0 ) i ca n't figure out what is wrong . i am testing it in the sandbox to no avail . xxmaj can some one help me please ? < ? php / / xxrep 60 - / / xxup paypal xxup posted xxup values xxup explanation / / https : / / xxunk / us / cgi - bin / ? xxunk - xxunk = developer / xxunk / / xxrep 60 - / / xxunk - xxmaj subscription end of term . a transaction type ( txn_type ) value . / / -",too localized,not a real question
3,"xxbos xxup gui xxmaj xxunk inconsistency / some valid moves work and others do not java i made a xxunk program that is a 2d array of buttons the valid moves are then enabled true and you can make a move ; some moves go xxunk while others just kind of sit there no matter how long you wait . i debugged and the methods are doing what they are supposed to but it does nt move . can anyone spot a problem i can not . import java.awt . * ; import java.awt.event . * ; import javax.swing . * ; public class xxunk extends japplet { private int score1 ; private int score2 ; private int turn ; private xxmaj icon p1 = new imageicon(""images / xxunk "" ) ; private xxmaj icon p2 = new imageicon(""images / xxunk "" ) ; private xxmaj icon xxunk = new",not a real question,not a real question
4,"xxbos detailsview loses datasource after callback with use icallbackeventhandler i am trying make in my asp.net application modifications without xxmaj postback and without updatepanel . xxmaj for paging and for modifying i want use only callbacks on the my site . xxmaj for it i use an interface "" icallbackeventhandler "" . xxmaj this simple application has "" xxunk "" & "" xxunk "" . xxmaj both with installed property xxunk "" . xxmaj but i have problem . i try save my detailsview with use javascript : xxunk xxup key ' ) xxmaj but my detailsview on server - side & client - htmlcode always * * empty * * ! xxmaj why ? xxmaj how i can transfer my edit of values from detailsview on server - side after user click the button "" save "" ? < % @ xxmaj page title=""home xxmaj page "" xxmaj language=""vb",open,open
5,"xxbos xxmaj how to generate unique numbers frm an array , dat too randomly i am doing a project in xxunk a

Path('models/high_level_API.pth')

### What's underneath the high level API

#### Training the Language Model

In [ ]:
## First we train a Language Model on the corpus
dls_lm = TextDataLoaders.from_df(df, is_lm=True, valid_pct=0.1)
wd = 0.1 # weight decay
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], wd=wd).to_fp16()
epochs, lr = 1, 1e-2
learn.fit_one_cycle(epochs, lr) # Trained on 1 cycle annealing
learn.save('1epoch')
learn = learn.load('1epoch')
learn.unfreeze() # unfreezing allows the body to be trained, freezing only allows the head to be trained
epochs, lr = 10, 1e-3
learn.fit_one_cycle(epochs, lr)
learn.save_encoder('finetuned') # finetuned on the unfrozen body, only the encoder part is saved.
learn.path = Path('./drive/MyDrive')
learn.save('finetuned')

#### Training the Text Classifier


In [ ]:
dls_clas = TextDataLoaders.from_df(df, valid_pct=0.1, text_vocab=dls_lm.vocab) # same vocab as dls_lm
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn = learn.load_encoder('finetuned') # load our finetuned Language Model as the encoder
### Gradual unfreezing from the end inwards gives better results than a full unfreeze.
learn.fit_one_cycle(1, 2e-2)
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))
learn.save('model')
learn.path = Path('./drive/MyDrive')
learn.save('model')